In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing.image import array_to_img
from random import seed
from random import randint
!mkdir images

In [ ]:
import sys
#sys.path.append("/content/drive/MyDrive/")
from load_models import *

In [2]:
def get_model(model_name,detection_type,augmentation=None):
    if(detection_type=="2_classes"):
        return nc_models[model_name]
    else:
        return ncl_models[model_name]

def load_image(img_path, show=True,img_size=224,rescale=True):
    img = image.load_img(img_path, target_size=(img_size, img_size))
    img_tensor = image.img_to_array(img)                    
    img_tensor = np.expand_dims(img_tensor, axis=0)
    if rescale==True:
        img_tensor /= 255.                                 
    if show:
        if rescale==True:
            plt.imshow(img_tensor[0])  
        else:
            new_im=img_tensor / 255. 
            plt.imshow(new_im[0]) 
        plt.axis('off')
        plt.show()
    return img_tensor

def prediction_classification(model_name,img_path,detection_type,augmentation=None):
  model = get_model(model_name,detection_type,augmentation)
  if(model_name=="InceptionV3"):
      image_size=299
  else:
      image_size=224
  if(model_name=="VGG16" or model_name=="VGG19") and (augmentation=="augmentation"):
      resc=False
  else:
      resc=True
  new_image = load_image(img_path,img_size=image_size,rescale=resc)
  # prediction
  pred = model.predict(new_image)
#     print(pred)
  res={}
  res['model_name']=str(model_name)
  if(model_name=="VGG16" or model_name=="VGG19") and (augmentation=="augmentation"):
      if(pred[0][0]>0.5):
          res['accuracy']=round(pred[0][0]*100,5)
          res['test']='un Tumeur detecté"'
      else:
          res['accuracy']=100-round(pred[0][0]*100,5)
          res['test']='Aucun Tumeur detecté'
  else:
      preds = np.argmax(pred, axis=1)
#         print(preds)
      if(preds==0):
          res['accuracy']=round(pred[0][0]*100,5)
          res['test']='Aucun Tumeur detecté'
      else:
          res['accuracy']=round(pred[0][1]*100,5)
          res['test']='un Tumeur detecté'
  print("Test: ",res['test'])
  print("Précision: ",res['accuracy'],"%")
  print("Modèle: ",res['model_name'])
  if detection_type=="mask_classification":
      print("Classification: avec masque")
  else:
      print("Classification: sans masque")
  if augmentation=="augmentation":
      print("Augmentation: Oui")
  elif augmentation=="no_augmentation":
      print("Augmentation: Non")
  # return res
